In [1]:
# import modules
import uproot, sys, time, math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
from scipy.special import betainc
from scipy.stats import norm

# import config functions
from jet_faking_plot_config import getWeight, zbi, sample_dict, getVarDict
from plot_var import variables, variables_data, ntuple_names, ntuple_names_BDT


# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 14.0,10.0  # Roughly 11 cm wde by 8 cm high
mpl.rcParams['font.size'] = 20.0 # Use 14 point font
sns.set(style="whitegrid")

font_size = {
    "xlabel": 17,
    "ylabel": 17,
    "xticks": 15,
    "yticks": 15,
    "legend": 14
}

plt.rcParams.update({
    "axes.labelsize": font_size["xlabel"],  # X and Y axis labels
    "xtick.labelsize": font_size["xticks"],  # X ticks
    "ytick.labelsize": font_size["yticks"],  # Y ticks
    "legend.fontsize": font_size["legend"]  # Legend
})

In [18]:
for i in range(len(tot2)):
    print(ntuple_names[i])
    print_cut(ntuple_names[i], tot2[i], "weighted after cut")

ggHyyd
Unweighted Events weighted after cut:  2059
Weighted Events weighted after cut:  208.62827
Zjets
Unweighted Events weighted after cut:  53
Weighted Events weighted after cut:  3.178844
Zgamma
Unweighted Events weighted after cut:  26918
Weighted Events weighted after cut:  906.9535
Wgamma
Unweighted Events weighted after cut:  17097
Weighted Events weighted after cut:  1713.7449
Wjets
Unweighted Events weighted after cut:  5038
Weighted Events weighted after cut:  1144.4331
gammajet_direct
Unweighted Events weighted after cut:  803
Weighted Events weighted after cut:  100.78318
data23
Unweighted Events weighted after cut:  1039
Weighted Events weighted after cut:  2090.6700000000096


In [21]:
208.62827 / np.sqrt(3.17 + 906.9535 + 1713.7449 + 1144.4331 + 100.78318 + 2090.67)

np.float64(2.7024580745916116)

In [2]:
tot = []
data = pd.DataFrame()
unweighted_bcut, weighted_bcut, unweighted_acut, weighted_acut = [], [], [], []
ntuple_names = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct', 'data23']

def test(fb):
    # checking if there are any none values
    mask = ak.is_none(fb['met_tst_et'])
    n_none = ak.sum(mask)
    print("Number of none values: ", n_none)
    # if n_none > 0:
    #     fb = fb[~mask]
    # print("Events after removing none values: ", len(fb), ak.sum(ak.is_none(fb['met_tst_et'])))

def print_cut(ntuple_name, fb, label):
    print(f"Unweighted Events {label}: ", len(fb))
    if ntuple_name == 'data23':
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name)))

for i in range(len(ntuple_names)):
    ucut, wcut = [], []
    start_time = time.time()
    ntuple_name = ntuple_names[i]
    if ntuple_name == 'data23': # data
        path = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/data23_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables_data, library="ak")
        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
                
        mask1 = (ak.firsts(fb['ph_topoetcone40'])-2450.)/ak.firsts(fb['ph_pt']) < 0.1   # jet_faking_photon cut
        fb = fb[mask1]
        fb = fb[fb['n_ph_baseline'] == 1]

    else: # MC
        path = f"/data/tmathew/ntups/mc23d/{ntuple_name}_y.root" 
        path_BDT = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/mc23d_{ntuple_name}_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables, library="ak")

        # add BDT score to fb
        f_BDT = uproot.open(path_BDT)['nominal']
        fb_BDT = f_BDT.arrays(["event", "BDTScore"], library="ak")
        tmp = fb["event"] == fb_BDT["event"]
        if np.all(tmp) == True:
            fb["BDTScore"] = fb_BDT["BDTScore"]
        else: 
            print("Something is wrong, need arranging")

        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
        fb = fb[fb['n_ph'] == 1]

    

    # Zjets and Wjets (rule out everything except for e->gamma)
    if ntuple_name == 'Zjets' or ntuple_name == 'Wjets':
        mask1 = ak.firsts(fb['ph_truth_type']) == 2
        mask2 = ak.firsts(fb['ph_truth_type']) == 2
        fb = fb[mask1 & mask2]
    
    print_cut(ntuple_name, fb, 'before cut')


    fb = fb[fb['n_mu_baseline'] == 0]
    fb = fb[fb['n_el_baseline'] == 0]
    fb = fb[fb['n_tau_baseline'] == 0]
    fb = fb[fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M']==1]
    fb = fb[ak.num(fb['ph_pt']) > 0] # prevent none values in Tbranch
    fb = fb[fb['met_tst_et'] >= 100000] # MET cut (basic cut)
    fb = fb[ak.firsts(fb['ph_pt']) >= 50000] # ph_pt cut (basic cut)
    fb = fb[fb['n_jet_central'] <= 4] # n_jet_central cut (basic cut)
    # goodPV on signal only
    if ntuple_name == 'ggHyyd':
        fb = fb[ak.num(fb['pv_z']) > 0]
        good_pv_tmp = (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5)
        fb = fb[good_pv_tmp]

    mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                            (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    mask1 = mt_tmp >= 100 # trigger cut
    fb = fb[mask1]

    fb = fb[fb['BDTScore'] >= 0.1] # added cut 1

    print_cut(ntuple_name, fb, 'after basic cut')

    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # metsig_tmp = fb['met_tst_sig'] # added cut 2 
    # mask1 = metsig_tmp >= 7
    # fb = fb[mask1]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # dphi_met_phterm_tmp = np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])) # added cut 3
    # fb = fb[dphi_met_phterm_tmp >= 1.35]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # dmet_tmp = fb['met_tst_noJVT_et'] - fb['met_tst_et'] # added cut 4
    # mask1 = dmet_tmp >= -20000
    # mask2 = dmet_tmp <= 50000
    # fb = fb[mask1 * mask2]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # dphi_met_jetterm_tmp = np.where(fb['met_jetterm_et'] != 0,   # added cut 5
    #                         np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
    #                         -999)
    # fb = fb[dphi_met_jetterm_tmp <= 0.7]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # ph_eta_tmp = np.abs(ak.firsts(fb['ph_eta'])) # added cut 6
    # fb = fb[ph_eta_tmp <= 1.75]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # phi1_tmp = ak.firsts(fb['jet_central_phi']) # added cut 7
    # phi2_tmp = ak.mask(fb['jet_central_phi'], ak.num(fb['jet_central_phi']) >= 2)[:, 1] 
    # dphi_tmp = np.arccos(np.cos(phi1_tmp - phi2_tmp))
    # dphi_jj_tmp = ak.fill_none(dphi_tmp, -999)
    # fb = fb[dphi_jj_tmp <= 2.5]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))


    # jet_sum_tmp = ak.sum(fb['jet_central_pt'], axis=-1)
    # expr = (fb['met_tst_et'] + ak.firsts(fb['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
    # balance_tmp = ak.where(jet_sum_tmp != 0, expr, 999) 
    # fb = fb[balance_tmp >= 0.65]
    # ucut.append(len(fb))
    # wcut.append(sum(getWeight(fb, ntuple_name)))

    # mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
    #                     (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    # mask1 = mt_tmp >= 95
    # fb = fb[mask1]

    # print_cut(ntuple_name, fb)
    ucut.append(len(fb))
    if ntuple_name == 'data23':
        wcut.append(sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        wcut.append(sum(getWeight(fb, ntuple_name)))


    unweighted_acut.append(ucut)
    weighted_acut.append(wcut)
    test(fb) # check for none value

    print(f"Reading Time for {ntuple_name}: {(time.time()-start_time)} seconds\n")



    tot.append(fb)

    fb = 0
    fb_BDT = 0
    tmp = 0


processing file:  /data/tmathew/ntups/mc23d/ggHyyd_y.root
Unweighted Events before cut:  195671
Weighted Events before cut:  19979.121
Unweighted Events after basic cut:  3729
Weighted Events after basic cut:  375.93835
Number of none values:  0
Reading Time for ggHyyd: 2.854734420776367 seconds

processing file:  /data/tmathew/ntups/mc23d/Zjets_y.root
Unweighted Events before cut:  3242488
Weighted Events before cut:  674497.9
Unweighted Events after basic cut:  5164
Weighted Events after basic cut:  124.29326
Number of none values:  0
Reading Time for Zjets: 99.14944171905518 seconds

processing file:  /data/tmathew/ntups/mc23d/Zgamma_y.root
Unweighted Events before cut:  3423357
Weighted Events before cut:  249515.86
Unweighted Events after basic cut:  704359
Weighted Events after basic cut:  16750.656
Number of none values:  0
Reading Time for Zgamma: 39.75036358833313 seconds

processing file:  /data/tmathew/ntups/mc23d/Wgamma_y.root
Unweighted Events before cut:  1308982
Weighted

In [25]:
375.93835 / np.sqrt(124.29326 + 16750.656 + 15736.289 + 14099.505 + 59503.31 + 125606.7300000043)

np.float64(0.7808011540031398)

In [9]:
def sel(tot):
    tot2 = []
    for i in range(len(tot)):
        fb2 = tot[i]

        jet_sum_tmp = ak.sum(fb2['jet_central_pt'], axis=-1)
        expr = (fb2['met_tst_et'] + ak.firsts(fb2['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
        balance_tmp = ak.where(jet_sum_tmp != 0, expr, -999)
        mask1 = balance_tmp >= 0.65
        mask2 = balance_tmp == -999
        fb2 = fb2[mask1 | mask2]
        
        metsig_tmp = fb2['met_tst_sig'] 
        mask1 = metsig_tmp >= 7
        fb2 = fb2[mask1]

        # mask2 = metsig_tmp <= 13
        # fb2 = fb2[mask1 * mask2]
        
        ph_eta_tmp = np.abs(ak.firsts(fb2['ph_eta']))
        fb2 = fb2[ph_eta_tmp <= 1.75]

        dphi_met_phterm_tmp = np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_phterm_phi'])) # added cut 3
        fb2 = fb2[dphi_met_phterm_tmp >= 1.55]

        dmet_tmp = fb2['met_tst_noJVT_et'] - fb2['met_tst_et']
        mask1 = dmet_tmp >= -20000
        mask2 = dmet_tmp <= 40000
        fb2 = fb2[mask1 * mask2]

        dphi_jj_tmp = fb2['dphi_central_jj']
        dphi_jj_tmp = ak.where(dphi_jj_tmp == -10, -999, dphi_jj_tmp)
        fb2 = fb2[dphi_jj_tmp <= 2.4]

        dphi_met_jetterm_tmp = np.where(fb2['met_jetterm_et'] != 0,   # added cut 5
                            np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_jetterm_phi'])),
                            -999)
        fb2 = fb2[dphi_met_jetterm_tmp <= 0.75]

        dphi_ph_centraljet1_tmp = np.arccos(np.cos(ak.firsts(fb2['ph_phi']) - ak.firsts(fb2['jet_central_phi'])))
        dphi_ph_centraljet1_tmp = ak.fill_none(dphi_ph_centraljet1_tmp, -999)
        mask1 = dphi_ph_centraljet1_tmp >= 1.8
        mask2 = dphi_ph_centraljet1_tmp == -999
        fb2 = fb2[mask1 | mask2]
        
        tot2.append(fb2)
    return tot2

tot2 = sel(tot)
# tot2 = tot

signal_name = 'ggHyyd'
cut_name = 'dphi_jj'

def getCutDict():
    cut_dict = {}

    cut_dict['BDTScore'] = {
        'lowercut': np.arange(0, 0.4+0.1, 0.1) # BDTScore > cut
    }
    cut_dict['balance'] = {
        'lowercut': np.arange(0, 1.5 + 0.05, 0.05), # balance > cut
        'uppercut': np.arange(5, 8 + 0.2, 0.2) # balance < cut
    }
    cut_dict['dmet'] = {
        'lowercut': np.arange(-30000, 0 + 5000, 5000), # dmet > cut
        'uppercut': np.arange(10000, 100000 + 5000, 5000), # -10000 < dmet < cut
    }
    cut_dict['dphi_jj'] = {
        'uppercut': np.arange(1, 3.1 + 0.1, 0.1) # dphi_jj < cut
    }
    cut_dict['dphi_met_jetterm'] = {
        'lowercut': np.arange(0, 1 + 0.05, 0.05), # dphi_met_jetterm > cut 
        'uppercut': np.arange(0.5, 2 + 0.05, 0.05), # dphi_met_jetterm < cut 
    }
    cut_dict['dphi_met_phterm'] = {
        'lowercut': np.arange(1, 2 + 0.05, 0.05), # dphi_met_phterm > cut
        'uppercut': np.arange(2, 3.1 + 0.1, 0.1), # dphi_met_phterm < cut
    }
    cut_dict['dphi_ph_centraljet1'] = {
        'lowercut': np.arange(0, 2.5 + 0.1, 0.1), # dphi_ph_centraljet1 > cut
        'uppercut': np.arange(1.5, 3.1 + 0.1, 0.1) # dphi_ph_centraljet1 < cut
    }
    cut_dict['dphi_phterm_jetterm'] = {
        'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
        'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
    }
    cut_dict['met'] = {
        'lowercut': np.arange(100000, 140000 + 5000, 5000),  # met > cut
        'uppercut': np.arange(140000, 300000 + 5000, 5000),  # met < cut
    }
    cut_dict['metsig'] = {
        'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
        'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
    }
    cut_dict['mt'] = {
        'lowercut': np.arange(80, 130+5, 5), # mt > cut
        'uppercut': np.arange(120, 230+5, 5) # mt < cut
    }
    cut_dict['n_jet_central'] = {
        'uppercut': np.arange(0, 8+1, 1) # njet < cut
    }
    cut_dict['ph_eta'] = {
        'uppercut': np.arange(1, 2.5 + 0.05, 0.05), # ph_eta < cut
    }
    cut_dict['ph_pt'] = {
        'lowercut': np.arange(50000, 100000 + 5000, 5000),  # ph_pt > cut
        'uppercut': np.arange(100000, 300000 + 10000, 10000),  # ph_pt > cut
    }

    return cut_dict
cut_config = getCutDict()

def calculate_significance(cut_var, cut_type, cut_values):
    sig_simple_list = []
    sig_s_plus_b_list = []
    sig_s_plus_1p3b_list = []
    sig_binomial_list = []

    sigacc_simple_list = []
    sigacc_s_plus_b_list = []
    sigacc_s_plus_1p3b_list = []
    sigacc_binomial_list = []

    acceptance_values = []  # Store acceptance percentages

    for cut in cut_values:
        sig_after_cut = 0
        bkg_after_cut = []
        sig_events = 0
        
        for i in range(len(ntuple_names)):
            fb = tot2[i]
            process = ntuple_names[i]
            var_config = getVarDict(fb, process, var_name=cut_var)
            x = var_config[cut_var]['var']
            mask = x != -999 # Apply cut: Remove -999 values 
            x = x[mask]

            if process == signal_name:
                sig_events = getWeight(fb, process)
                sig_events = sig_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                sig_after_cut = ak.sum(sig_events[mask])
            
            else:
                bkg_events = getWeight(fb, process)
                bkg_events = bkg_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                bkg_after_cut.append(ak.sum(bkg_events[mask]))

       # Now compute different types of significance
        total_bkg = sum(bkg_after_cut)
        total_signal = sig_after_cut

        # Avoid zero division carefully
        if total_bkg > 0:
            sig_simple = total_signal / np.sqrt(total_bkg)
            sig_s_plus_b = total_signal / np.sqrt(total_signal + total_bkg) if (total_signal + total_bkg) > 0 else 0
            sig_s_plus_1p3b = total_signal / np.sqrt(total_signal + 1.3 * total_bkg) if (total_signal + 1.3*total_bkg) > 0 else 0
            sig_binomial = zbi(total_signal, total_bkg, sigma_b_frac=0.3)
        else:
            sig_simple = sig_s_plus_b = sig_s_plus_1p3b = sig_binomial = 0

        # Acceptance
        acceptance = total_signal / sum(sig_events) if sum(sig_events) > 0 else 0
        acceptance_values.append(acceptance * 100)  # percentage

        # Save significance
        sig_simple_list.append(sig_simple)
        sig_s_plus_b_list.append(sig_s_plus_b)
        sig_s_plus_1p3b_list.append(sig_s_plus_1p3b)
        sig_binomial_list.append(sig_binomial)

        # Save significance × acceptance
        sigacc_simple_list.append(sig_simple * acceptance)
        sigacc_s_plus_b_list.append(sig_s_plus_b * acceptance)
        sigacc_s_plus_1p3b_list.append(sig_s_plus_1p3b * acceptance)
        sigacc_binomial_list.append(sig_binomial * acceptance)

    return (sig_simple_list, sig_s_plus_b_list, sig_s_plus_1p3b_list, sig_binomial_list,
            sigacc_simple_list, sigacc_s_plus_b_list, sigacc_s_plus_1p3b_list, sigacc_binomial_list,
            acceptance_values)

# Compute significance for each variable dynamically
for cut_var, cut_types in cut_config.items():
    for cut_type, cut_values in cut_types.items():
        (sig_simple_list, sig_s_plus_b_list, sig_s_plus_1p3b_list, sig_binomial_list,
         sigacc_simple_list, sigacc_s_plus_b_list, sigacc_s_plus_1p3b_list, sigacc_binomial_list,
         acceptance_values) = calculate_significance(cut_var, cut_type, cut_values)

        # Plot results
        fig, (ax_top, ax_bot) = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

        # Top plot: Significance vs. Cut
        ax_top.plot(cut_values, sig_simple_list, marker='o', label='S/√B')
        # ax_top.plot(cut_values, sig_s_plus_b_list, marker='s', label='S/√(S+B)')
        # ax_top.plot(cut_values, sig_s_plus_1p3b_list, marker='^', label='S/√(S+1.3B)')
        # ax_top.plot(cut_values, sig_binomial_list, marker='x', label='BinomialExpZ')
        ax_top.set_ylabel('Significance')
        ax_top.set_title(f'Significance vs. {cut_var} ({cut_type})')
        ax_top.legend()
        ax_top.grid(True)

        # Bottom plot: Significance * Acceptance vs. Cut
        ax_bot.plot(cut_values, sigacc_simple_list, marker='o', label='(S/√B) × Acceptance')
        # ax_bot.plot(cut_values, sigacc_s_plus_b_list, marker='s', label='(S/√(S+B)) × Acceptance')
        # ax_bot.plot(cut_values, sigacc_s_plus_1p3b_list, marker='^', label='(S/√(S+1.3B)) × Acceptance')
        # ax_bot.plot(cut_values, sigacc_binomial_list, marker='x', label='BinomialExpZ × Acceptance')

        for i, txt in enumerate(acceptance_values):
            ax_bot.text(cut_values[i], sigacc_simple_list[i], f'{txt:.1f}%', 
                        fontsize=10, ha='right', va='bottom', color='purple')
            
        ax_bot.set_xlabel(f'{cut_var} Cut')
        ax_bot.set_ylabel('Significance × Acceptance')
        ax_bot.set_title(f'Significance × Acceptance vs. {cut_var} ({cut_type})')
        
        ax_bot.set_xticks(cut_values)
        ax_bot.set_xticklabels(ax_bot.get_xticks(), rotation=45, ha='right')
        ax_bot.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        # ax_bot.xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))  # Show at most 10 x-ticks
        
        var_configs_tmp = getVarDict(tot2[0], signal_name, cut_var)
        ax_bot.set_xlabel(var_configs_tmp[cut_var]['title'])
        ax_bot.legend()
        ax_bot.grid(True)

        plt.tight_layout()
        plt.savefig(f"../jets_faking_photons/lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
        print(f"Successfully saved to ../jets_faking_photons/lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
        plt.close()

Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_BDTScore_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_balance_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_balance_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_dmet_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_dmet_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_dphi_jj_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_dphi_met_jetterm_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_dphi_met_jetterm_uppercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_dphi_jjcut/significance_dphi_met_phterm_lowercut.png
Successfully saved to ../jets_faking_photon

In [22]:
n_1_config = ["dphi_met_phterm", "metsig", "ph_eta", "dmet", "dphi_met_jetterm",  "dphi_ph_centraljet1", "balance", "dphi_jj"]

def sel(tot, n_1_name=None):
    tot2 = []
    for i in range(len(tot)):
        fb2 = tot[i]
    
        if n_1_name != "dphi_met_phterm":
            dphi_met_phterm_tmp = np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_phterm_phi']))
            fb2 = fb2[dphi_met_phterm_tmp >= 1.35]

        if n_1_name != "balance":
            jet_sum_tmp = ak.sum(fb2['jet_central_pt'], axis=-1)
            expr = (fb2['met_tst_et'] + ak.firsts(fb2['ph_pt'])) / ak.where(jet_sum_tmp != 0, jet_sum_tmp, 1)
            balance_tmp = ak.where(jet_sum_tmp != 0, expr, -999)
            mask1 = balance_tmp >= 0.70
            mask2 = balance_tmp == -999
            fb2 = fb2[mask1 | mask2]

        if n_1_name != "metsig":
            metsig_tmp = fb2['met_tst_sig'] 
            mask1 = metsig_tmp >= 6
            # fb2 = fb2[mask1]
            mask2 = metsig_tmp <= 16
            fb2 = fb2[mask1 * mask2]
        
        if n_1_name != "ph_eta":
            ph_eta_tmp = np.abs(ak.firsts(fb2['ph_eta']))
            fb2 = fb2[ph_eta_tmp <= 1.75]

        if n_1_name != "dmet":
            dmet_tmp = fb2['met_tst_noJVT_et'] - fb2['met_tst_et']
            mask1 = dmet_tmp >= -20000
            mask2 = dmet_tmp <= 40000
            fb2 = fb2[mask1 * mask2]

        if n_1_name != "dphi_met_jetterm":
            dphi_met_jetterm_tmp = np.where(fb2['met_jetterm_et'] != 0,   # added cut 5
                                    np.arccos(np.cos(fb2['met_tst_phi'] - fb2['met_jetterm_phi'])),
                                    -999)
            fb2 = fb2[dphi_met_jetterm_tmp <= 0.65]

        if n_1_name != "dphi_jj":
            dphi_jj_tmp = fb2['dphi_central_jj']
            dphi_jj_tmp = ak.where(dphi_jj_tmp == -10, -999, dphi_jj_tmp)
            fb2 = fb2[dphi_jj_tmp <= 2.4]

        if n_1_name != "dphi_ph_centraljet1":
            dphi_ph_centraljet1_tmp = np.arccos(np.cos(ak.firsts(fb2['ph_phi']) - ak.firsts(fb2['jet_central_phi'])))
            dphi_ph_centraljet1_tmp = ak.fill_none(dphi_ph_centraljet1_tmp, -999)
            mask1 = dphi_ph_centraljet1_tmp >= 1.7
            mask2 = dphi_ph_centraljet1_tmp == -999
            fb2 = fb2[mask1 | mask2]
        
        tot2.append(fb2)
    return tot2

tot2 = sel(tot)
# tot2 = tot

signal_name = 'ggHyyd'  # Define signal dataset
cut_name = 'n-1'

def getCutDict(n_1_name=None):
    cut_dict = {}

    if n_1_name is None or n_1_name == "BDTScore":
        cut_dict['BDTScore'] = {
            'lowercut': np.arange(0, 0.4+0.1, 0.1) # BDTScore > cut
        }
    if n_1_name is None or n_1_name == "balance":
        cut_dict['balance'] = {
            'lowercut': np.arange(0, 1.5 + 0.05, 0.05), # balance > cut
            'uppercut': np.arange(5, 8 + 0.2, 0.2) # balance < cut
        }
    if n_1_name is None or n_1_name == "dmet":
        cut_dict['dmet'] = {
            'lowercut': np.arange(-30000, 0 + 5000, 5000), # dmet > cut
            'uppercut': np.arange(10000, 100000 + 5000, 5000), # -10000 < dmet < cut
        }
    if n_1_name is None or n_1_name == "dphi_jj":
        cut_dict['dphi_jj'] = {
            'uppercut': np.arange(1, 3.1 + 0.1, 0.1) # dphi_jj < cut
        }
    # if n_1_name is None or n_1_name == "dphi_met_phterm_minus_dphi_met_jetterm":
    #     cut_dict['dphi_met_phterm_minus_dphi_met_jetterm'] = {
    #         'lowercut': np.arange(0, 1.5+0.05, 0.05),
    #         'uppercut': np.arange(1.5, 3.1+0.05, 0.05)
    #     }
    if n_1_name is None or n_1_name == "dphi_met_jetterm":
        cut_dict['dphi_met_jetterm'] = {
            'lowercut': np.arange(0, 1 + 0.05, 0.05), # dphi_met_jetterm > cut 
            'uppercut': np.arange(0.5, 2 + 0.05, 0.05), # dphi_met_jetterm < cut 
        }
    if n_1_name is None or n_1_name == "dphi_met_phterm":
        cut_dict['dphi_met_phterm'] = {
            'lowercut': np.arange(1, 2 + 0.05, 0.05), # dphi_met_phterm > cut
            'uppercut': np.arange(2, 3.1 + 0.1, 0.1), # dphi_met_phterm < cut
        }
    if n_1_name is None or n_1_name == "dphi_ph_centraljet1":
        cut_dict['dphi_ph_centraljet1'] = {
            'lowercut': np.arange(0, 2.5 + 0.1, 0.1), # dphi_ph_centraljet1 > cut
            'uppercut': np.arange(1.5, 3.1 + 0.1, 0.1) # dphi_ph_centraljet1 < cut
        }
    if n_1_name is None or n_1_name == "dphi_phterm_jetterm":
        cut_dict['dphi_phterm_jetterm'] = {
            'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
            'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
        }
    if n_1_name is None or n_1_name == "met":
        cut_dict['met'] = {
            'lowercut': np.arange(100000, 140000 + 5000, 5000),  # met > cut
            'uppercut': np.arange(140000, 300000 + 5000, 5000),  # met < cut
        }
    if n_1_name is None or n_1_name == "metsig":
        cut_dict['metsig'] = {
            'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
            'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
        }
    if n_1_name is None or n_1_name == "mt":
        cut_dict['mt'] = {
            'lowercut': np.arange(80, 130+5, 5), # mt > cut
            'uppercut': np.arange(120, 230+5, 5) # mt < cut
        }
    if n_1_name is None or n_1_name == "n_jet_central":
        cut_dict['n_jet_central'] = {
            'uppercut': np.arange(0, 8+1, 1) # njet < cut
        }
    if n_1_name is None or n_1_name == "ph_eta":
        cut_dict['ph_eta'] = {
            'uppercut': np.arange(1, 2.5 + 0.05, 0.05), # ph_eta < cut
        }
    if n_1_name is None or n_1_name == "ph_pt":
        cut_dict['ph_pt'] = {
            'lowercut': np.arange(50000, 100000 + 5000, 5000),  # ph_pt > cut
            'uppercut': np.arange(100000, 300000 + 10000, 10000),  # ph_pt > cut
        }

    return cut_dict


def calculate_significance(cut_var, cut_type, cut_values):
    sig_simple_list = []
    sig_s_plus_b_list = []
    sig_s_plus_1p3b_list = []
    sig_binomial_list = []

    sigacc_simple_list = []
    sigacc_s_plus_b_list = []
    sigacc_s_plus_1p3b_list = []
    sigacc_binomial_list = []

    acceptance_values = []  # Store acceptance percentages

    for cut in cut_values:
        sig_after_cut = 0
        bkg_after_cut = []
        sig_events = 0
        
        for i in range(len(ntuple_names)):
            fb = tot2[i]
            process = ntuple_names[i]
            var_config = getVarDict(fb, process, var_name=cut_var)
            x = var_config[cut_var]['var']
            mask = x != -999 # Apply cut: Remove -999 values 
            x = x[mask]

            if process == signal_name:
                sig_events = getWeight(fb, process)
                sig_events = sig_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                sig_after_cut = ak.sum(sig_events[mask])
            
            else:
                bkg_events = getWeight(fb, process)
                bkg_events = bkg_events[mask]
                if cut_type == 'lowercut':
                    mask = x >= cut
                elif cut_type == 'uppercut':
                    mask = x <= cut
                else:
                    raise ValueError("Invalid cut type")
                bkg_after_cut.append(ak.sum(bkg_events[mask]))

       # Now compute different types of significance
        total_bkg = sum(bkg_after_cut)
        total_signal = sig_after_cut

        # Avoid zero division carefully
        if total_bkg > 0:
            sig_simple = total_signal / np.sqrt(total_bkg)
            sig_s_plus_b = total_signal / np.sqrt(total_signal + total_bkg) if (total_signal + total_bkg) > 0 else 0
            sig_s_plus_1p3b = total_signal / np.sqrt(total_signal + 1.3 * total_bkg) if (total_signal + 1.3*total_bkg) > 0 else 0
            sig_binomial = zbi(total_signal, total_bkg, sigma_b_frac=0.3)
        else:
            sig_simple = sig_s_plus_b = sig_s_plus_1p3b = sig_binomial = 0

        # Acceptance
        acceptance = total_signal / sum(sig_events) if sum(sig_events) > 0 else 0
        acceptance_values.append(acceptance * 100)  # percentage

        # Save significance
        sig_simple_list.append(sig_simple)
        sig_s_plus_b_list.append(sig_s_plus_b)
        sig_s_plus_1p3b_list.append(sig_s_plus_1p3b)
        sig_binomial_list.append(sig_binomial)

        # Save significance × acceptance
        sigacc_simple_list.append(sig_simple * acceptance)
        sigacc_s_plus_b_list.append(sig_s_plus_b * acceptance)
        sigacc_s_plus_1p3b_list.append(sig_s_plus_1p3b * acceptance)
        sigacc_binomial_list.append(sig_binomial * acceptance)

    return (sig_simple_list, sig_s_plus_b_list, sig_s_plus_1p3b_list, sig_binomial_list,
            sigacc_simple_list, sigacc_s_plus_b_list, sigacc_s_plus_1p3b_list, sigacc_binomial_list,
            acceptance_values)

for cut_var_tmp in n_1_config:
    cut_config = getCutDict(n_1_name=cut_var_tmp)
    tot2 = sel(tot, n_1_name=cut_var_tmp)
    for cut_var, cut_types in cut_config.items():
        for cut_type, cut_values in cut_types.items():
            (sig_simple_list, sig_s_plus_b_list, sig_s_plus_1p3b_list, sig_binomial_list,
            sigacc_simple_list, sigacc_s_plus_b_list, sigacc_s_plus_1p3b_list, sigacc_binomial_list,
            acceptance_values) = calculate_significance(cut_var, cut_type, cut_values)

            # Plot results
            fig, (ax_top, ax_bot) = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

            # Top plot: Significance vs. Cut
            ax_top.plot(cut_values, sig_simple_list, marker='o', label='S/√B')
            # ax_top.plot(cut_values, sig_s_plus_b_list, marker='s', label='S/√(S+B)')
            # ax_top.plot(cut_values, sig_s_plus_1p3b_list, marker='^', label='S/√(S+1.3B)')
            # ax_top.plot(cut_values, sig_binomial_list, marker='x', label='BinomialExpZ')
            ax_top.set_ylabel('Significance')
            ax_top.set_title(f'Significance vs. {cut_var} ({cut_type})')
            ax_top.legend()
            ax_top.grid(True)

            # Bottom plot: Significance * Acceptance vs. Cut
            ax_bot.plot(cut_values, sigacc_simple_list, marker='o', label='(S/√B) × Acceptance')
            # ax_bot.plot(cut_values, sigacc_s_plus_b_list, marker='s', label='(S/√(S+B)) × Acceptance')
            # ax_bot.plot(cut_values, sigacc_s_plus_1p3b_list, marker='^', label='(S/√(S+1.3B)) × Acceptance')
            # ax_bot.plot(cut_values, sigacc_binomial_list, marker='x', label='BinomialExpZ × Acceptance')

            for i, txt in enumerate(acceptance_values):
                ax_bot.text(cut_values[i], sigacc_simple_list[i], f'{txt:.1f}%', 
                            fontsize=10, ha='right', va='bottom', color='purple')
                
            ax_bot.set_xlabel(f'{cut_var} Cut')
            ax_bot.set_ylabel('Significance × Acceptance')
            ax_bot.set_title(f'Significance × Acceptance vs. {cut_var} ({cut_type})')
            
            ax_bot.set_xticks(cut_values)
            ax_bot.set_xticklabels(ax_bot.get_xticks(), rotation=45, ha='right')
            ax_bot.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
            # ax_bot.xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))  # Show at most 10 x-ticks
            
            var_configs_tmp = getVarDict(tot2[0], signal_name, cut_var)
            ax_bot.set_xlabel(var_configs_tmp[cut_var]['title'])
            ax_bot.legend()
            ax_bot.grid(True)

            plt.tight_layout()
            plt.savefig(f"../jets_faking_photons/lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
            print(f"Successfully saved to ../jets_faking_photons/lumi135/mc23d_{cut_name}cut/significance_{cut_var}_{cut_type}.png")
            plt.close()


    var_config = getVarDict(tot2[0], 'ggHyyd', var_name=cut_var_tmp)

    for var in var_config:
        # print(var)
        bg_values = []     
        bg_weights = []    
        bg_colors = []     
        bg_labels = []     

        signal_values = [] 
        signal_weights = []
        signal_color = None 
        signal_label = None

        for j in range(len(ntuple_names)):
            process = ntuple_names[j]
            fb = tot2[j]  # TTree
            var_config = getVarDict(fb, process, var_name=var)

            x = var_config[var]['var'] # TBranch
            bins = var_config[var]['bins'] 

            if 'weight' in var_config[var]:  # If weight is there
                weights = var_config[var]['weight']
            else:
                weights = getWeight(fb, process)
            
            sample_info = sample_dict[process]
            color = sample_info['color']
            legend = sample_info['legend']

            
            if process == 'ggHyyd':  # signal
                signal_values.append(x)
                signal_weights.append(weights)
                signal_color = color
                signal_label = legend
            else:   # background
                bg_values.append(x)
                bg_weights.append(weights)
                bg_colors.append(color)
                bg_labels.append(legend)

        fig, (ax_top, ax_bot) = plt.subplots(2, 1, figsize=(12, 13), gridspec_kw={'height_ratios': [9, 4]})

        ax_top.hist(bg_values, bins=bins, weights=bg_weights, color=bg_colors,
                    label=bg_labels, stacked=True)

        ax_top.hist(signal_values, bins=bins, weights=signal_weights, color=signal_color,
                    label=signal_label, histtype='step', linewidth=2)

        signal_all = np.concatenate(signal_values) if len(signal_values) > 0 else np.array([])
        signal_weights_all = np.concatenate(signal_weights) if len(signal_weights) > 0 else np.array([])

        # Add error bar for signal (top plot)
        if len(signal_all) > 0:
            signal_counts, bin_edges = np.histogram(signal_all, bins=bins, weights=signal_weights_all)
            sum_weights_sq, _ = np.histogram(signal_all, bins=bins, weights=signal_weights_all**2)
            bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
            signal_errors = np.sqrt(sum_weights_sq)  # Poisson error sqrt(N)

            ax_top.errorbar(bin_centers, signal_counts, yerr=signal_errors, fmt='.', linewidth=2,
                            color=signal_color, capsize=0)

        ax_top.set_yscale('log')
        ax_top.set_ylim(0.0001, 1e11)
        ax_top.set_xlim(bins[0], bins[-1])
        ax_top.minorticks_on()
        ax_top.grid(True, which="both", linestyle="--", linewidth=0.5)
        ax_top.set_ylabel("Events")
        ax_top.legend(ncol=2)
        # ax_top.set_title("vtx_sumPt distribution")

        bg_all = np.concatenate(bg_values) if len(bg_values) > 0 else np.array([])
        bg_weights_all = np.concatenate(bg_weights) if len(bg_weights) > 0 else np.array([])

        # Compute the weighted histogram counts using np.histogram
        S_counts, _ = np.histogram(signal_all, bins=bins, weights=signal_weights_all)
        B_counts, _ = np.histogram(bg_all, bins=bins, weights=bg_weights_all)     

        # Compute per-bin significance
        sig_simple = np.zeros_like(S_counts, dtype=float)
        sig_s_plus_b = np.zeros_like(S_counts, dtype=float)
        sig_s_plus_1p3b = np.zeros_like(S_counts, dtype=float)

        sqrt_B = np.sqrt(B_counts)
        sqrt_SplusB = np.sqrt(S_counts + B_counts)
        sqrt_Splus1p3B = np.sqrt(S_counts + 1.3 * B_counts)

        # Avoid division by zero safely
        sig_simple = np.where(B_counts > 0, S_counts / sqrt_B, 0)
        sig_s_plus_b = np.where((S_counts + B_counts) > 0, S_counts / sqrt_SplusB, 0)
        sig_s_plus_1p3b = np.where((S_counts + 1.3 * B_counts) > 0, S_counts / sqrt_Splus1p3B, 0)

        # Add Binomial ExpZ per bin
        zbi_per_bin = np.array([
            zbi(S_counts[i], B_counts[i], sigma_b_frac=0.3)
            for i in range(len(S_counts))
        ])

        # Compute the bin centers for plotting
        bin_centers = 0.5 * (bins[:-1] + bins[1:])

        # Compute the total significance: total S / sqrt(total B)
        total_signal = np.sum(S_counts)
        total_bkg = np.sum(B_counts)

        if total_bkg > 0:
            total_sig_simple = total_signal / np.sqrt(total_bkg)
            total_sig_s_plus_b = total_signal / np.sqrt(total_signal + total_bkg)
            total_sig_s_plus_1p3b = total_signal / np.sqrt(total_signal + 1.3 * total_bkg)
            total_sig_binomial = zbi(total_signal, total_bkg, sigma_b_frac=0.3)
        else:
            total_sig_simple = total_sig_s_plus_b = total_sig_s_plus_1p3b = total_sig_binomial = 0

        # --- Plot all significance curves ---
        ax_bot.step(bin_centers, sig_simple, where='mid', color='chocolate', linewidth=2,
                    label=f"S/√B = {total_sig_simple:.4f}")
        ax_bot.step(bin_centers, sig_s_plus_b, where='mid', color='tomato', linewidth=2,
                    label=f"S/√(S+B) = {total_sig_s_plus_b:.4f}")
        ax_bot.step(bin_centers, sig_s_plus_1p3b, where='mid', color='orange', linewidth=2,
                    label=f"S/√(S+1.3B) = {total_sig_s_plus_1p3b:.4f}")
        ax_bot.step(bin_centers, zbi_per_bin, where='mid', color='plum', linewidth=2,
                    label=f"Binomial ExpZ = {total_sig_binomial:.4f}")

        ax_bot.set_xlabel(var_config[var]['title'])
        # ax_bot.set_xticks(np.linspace(bins[0], bins[-1], 11))
        ax_bot.set_ylabel("Significance")
        ax_bot.set_ylim(-0.8, 2)
        ax_top.set_xlim(bins[0], bins[-1])

        # Do not set a title on the bottom plot.
        ax_bot.set_title("")

        # Draw a legend with purple text.
        leg = ax_bot.legend()
        for text in leg.get_texts():
            text.set_color('purple')

        plt.xlim(bins[0], bins[-1])
        plt.tight_layout()
        plt.savefig(f"../jets_faking_photons/lumi135/mc23d_{cut_name}cut/{var}_nodijet.png")
        print(f"successfully saved to ../jets_faking_photons/lumi135/mc23d_{cut_name}cut/{var}_nodijet.png")
        plt.close()
        # plt.show()

        y_true = np.concatenate([np.ones_like(signal_all), np.zeros_like(bg_all)])
        # Use the vtx_sumPt values as the classifier output.
        y_scores = np.concatenate([signal_all, bg_all])
        # Combine the weights for all events.
        y_weights = np.concatenate([signal_weights_all, bg_weights_all])

        # Compute the weighted ROC curve.
        fpr, tpr, thresholds = roc_curve(y_true, y_scores, sample_weight=y_weights)
        sorted_indices = np.argsort(fpr)
        fpr_sorted = fpr[sorted_indices]
        tpr_sorted = tpr[sorted_indices]

        roc_auc = auc(fpr_sorted, tpr_sorted)

        # Create a new figure for the ROC curve.
        plt.figure(figsize=(8, 8))
        plt.plot(fpr, tpr, lw=2, color='red', label=f'ROC curve (AUC = {roc_auc:.5f})')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random chance')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve for {var}")
        plt.legend(loc="lower right")
        plt.grid(True, which="both", linestyle="--", linewidth=0.5)
        plt.tight_layout()    
        plt.savefig(f"../jets_faking_photons/lumi135/mc23d_{cut_name}cut/roc_curve_{var}.png")
        print(f"successfully saved to ../jets_faking_photons/lumi135/mc23d_{cut_name}cut/roc_curve_{var}.png")
        plt.close()
        # plt.show()


Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_dphi_met_phterm_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_dphi_met_phterm_uppercut.png


/home/jlai/.local/lib/python3.9/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))


successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/dphi_met_phterm_nodijet.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/roc_curve_dphi_met_phterm.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_metsig_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_metsig_uppercut.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/metsig_nodijet.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/roc_curve_metsig.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_ph_eta_uppercut.png


/home/jlai/.local/lib/python3.9/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))


successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/ph_eta_nodijet.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/roc_curve_ph_eta.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_dmet_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_dmet_uppercut.png


...

successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/dphi_ph_centraljet1_nodijet.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/roc_curve_dphi_ph_centraljet1.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_balance_lowercut.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_balance_uppercut.png


/home/jlai/.local/lib/python3.9/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in sqrt
  return impl(*broadcasted_args, **(kwargs or {}))
/home/jlai/.local/lib/python3.9/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))


successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/balance_nodijet.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/roc_curve_balance.png
Successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/significance_dphi_jj_uppercut.png


/home/jlai/.local/lib/python3.9/site-packages/awkward/_nplikes/array_module.py:251: RuntimeWarning: invalid value encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))


successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/dphi_jj_nodijet.png
successfully saved to ../jets_faking_photons/lumi135/mc23d_n-1cut/roc_curve_dphi_jj.png
